presentation day test 

In [9]:
import numpy as np
import scipy.io
import sklearn.metrics
import sklearn 
import os
import random
import pandas as pd
import time
# sys.path.append('C:/Users/liqia/Spring2021-Project3-group-3-master/lib/')

In [10]:
from scipy.spatial.distance import pdist
import imbalanced_databases as imbd
import matplotlib.pyplot as plt
# import smote_variants as s
import scipy.io as scio
from PIL import Image
import pandas as pd
import numpy as np
import xlsxwriter
import scipy.io
import sklearn
import os, sys
import pickle
import random
import time
import cv2

from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from sklearn.metrics import (classification_report, 
                            confusion_matrix, 
                            recall_score,
                            accuracy_score,
                            make_scorer,
                            roc_auc_score)
from sklearn.model_selection import (train_test_split,
                                    cross_validate, 
                                    GridSearchCV,
                                    RepeatedStratifiedKFold,
                                    cross_val_score,
                                    validation_curve)
from imblearn.under_sampling import RandomUnderSampler               
from imblearn.over_sampling import SMOTE
# models
from sklearn.ensemble import (GradientBoostingClassifier, 
                              AdaBoostClassifier,
                             VotingClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


In [13]:
#3 columns: index, baseline, advanced
index=list(range(0,3000))
test_df=pd.DataFrame({'index':index})
test_df['baseline']=""
test_df['advanced']=""
# test_df.to_csv('C:/Users/liqia/Spring2021-Project3-group-3-master/data/label_prediction_test.csv',index=False)
test_df.to_csv('C:/Users/morrismr/Documents/Spring2021-Project3-group-3-master/data/label_prediction_test.csv',index=False)

In [15]:
# results = pd.read_csv('C:/Users/liqia/Spring2021-Project3-group-3-master/data/label_prediction_test.csv')
results = pd.read_csv('C:/Users/morrismr/Documents/Spring2021-Project3-group-3-master/data/label_prediction_test.csv')
results.head()

,index,baseline,advanced
0,0,NaN,NaN
1,1,NaN,NaN
2,2,NaN,NaN
3,3,NaN,NaN
4,4,NaN,NaN


In [16]:
# path = "C:/Users/liqia/Spring2021-Project3-group-3-master/data//train_set"
path = "C:/Users/morrismr/Documents/Spring2021-Project3-group-3-master/data//train_set"
image_dir = path+"images/"
pt_dir = path+"points/"
label_path = path+"label.csv"

In [22]:
def flatten(x):
    result = []
    for el in x:
        result.append(el[0])
        result.append(el[1])
    return result

In [24]:

fid = pd.read_csv(r'C:\Users\morrismr\Downloads\train_set\train_set\label.csv')
label = np.array(fid['label'])

# dataFile = 'C:\\Users\\morrismr\\Downloads\\train_set\\train_set\\points\\'
dataFile = 'C:\\Users\\morrismr\\Documents\\Spring2021-Project3-group-3-master\\data\\train_set\\points\\'
img_set = []
for i in range(1, 3001):
    if i < 10:
        img_name = '000' + str(i)
    elif i < 100:
        img_name = '00' + str(i)
    elif i < 1000:
        img_name = '0' + str(i)
    else:
        img_name = str(i)
    image = scio.loadmat(dataFile + img_name + '.mat')

    try:
        image = image['faceCoordinatesUnwarped']
    except KeyError as ke:
        image = image['faceCoordinates2']

    image = np.array(flatten(image))
    img_set.append(image)
img_set = np.array(img_set)
print(img_set.shape)

X_original = img_set.copy()
y_original = label.copy()
print('loaded original version')

(3000, 156)
loaded original version


In [17]:
def read_data(path):
  
  # read labels
  labels = pd.read_csv(path+'/label.csv')
  y= labels['label'].to_numpy()

  # read points
  n = 3000
  for i in range(1,n+1):
    p_path = str(i).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords = mat['faceCoordinatesUnwarped'] 
    else:
      cords = mat['faceCoordinates2']

    distance = sklearn.metrics.pairwise_distances(cords)       
          # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords[:,0]), k = 1)]    
          # stretch the upper triangle of the symmetric matrix 
          # to a long array with dimension 3003
          # 3003 = (1+77)*78/2
    if i==1:
      distances = np.mat([flatten_distance])
    else:
      distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
  return (distances, y)

In [18]:
read_time_start=time.time()
X, Y = read_data(path)
print(time.time() - read_time_start)

48.46916103363037


In [27]:
baseline = pickle.load(open("C:/Users/morrismr/Documents/Spring2021-Project3-group-3-master/output/baseline.pkl", "rb"))
baseline_preds = baseline.predict(X_original)
results['baseline'] = list(baseline_preds)
print(results)

      index  baseline  advanced
0         0         0       NaN
1         1         0       NaN
2         2         0       NaN
3         3         0       NaN
4         4         0       NaN
...     ...       ...       ...
2995   2995         0       NaN
2996   2996         0       NaN
2997   2997         0       NaN
2998   2998         1       NaN
2999   2999         0       NaN

[3000 rows x 3 columns]


In [36]:
advanced = pickle.load(open("C:/Users/morrismr/Documents/Spring2021-Project3-group-3-master/output/advanced.pkl", "rb"))
advanced_preds = advanced.predict(X)
results['advanced'] = list(advanced_preds)
print(results)

      index  baseline  advanced
0         0         0         0
1         1         0         0
2         2         0         0
3         3         0         0
4         4         0         0
...     ...       ...       ...
2995   2995         0         0
2996   2996         0         0
2997   2997         0         0
2998   2998         1         1
2999   2999         0         0

[3000 rows x 3 columns]


In [37]:
results.to_csv('C:/Users/morrismr/Documents/Spring2021-Project3-group-3-master/output/label_prediction.csv',index=False)

In [38]:
results.head(10)

,index,baseline,advanced
0,0,0,0
1,1,0,0
2,2,0,0
3,3,0,0
4,4,0,0
5,5,1,1
6,6,0,1
7,7,0,1
8,8,1,1
9,9,0,0
